# (day3) Section 2: LGTM ( Long Short Term Memory )

本書は、「深層学習後編（day3）レポート」の、「Section 2: LGTM」についてのものです。


## 1. 要点まとめ


<!-- - page. 55 - 83

- Section 2: LGTM
  - RNN の課題
  - 勾配爆発
    - 勾配のクリッピング
  - LSTM の全体図
  - 2-1 CEC
    - 数式
    - 課題
  - 2-2 入力ゲートと出力ゲート
    - 入力ゲート
    - 出力ゲート
    - CEC の課題解決
  - 2-3 忘却ゲート
    - LSTM ブロックの課題
  - 2-4 覗き穴結合
    - CEC の課題解決 -->


### RNN の課題

RNN では、時系列 ( 時間的順序 ) を遡る。よって、その過程で ( NN よりさらに ) 勾配を消失しやすい。

これを構造によって、解決したものが LSTM である。


### 勾配爆発

勾配消失は、勾配が 0 に近づいていくことに対して、<br>
勾配爆発は、逆伝播の過程で勾配が指数関数的に大きくなってしまうこと。
( 勾配が発散してしまう )


#### 勾配のクリッピング

勾配爆発を防ぐための手法。<br>
勾配のノルムがしきい値を超えたら、勾配のノルムをしきい値に正規化する。

勾配のしきい値を $ M $ 、勾配を $ g $ として、以下の上限値に正規化する。

$$
g = \frac{M}{\Vert g \Vert} g
$$

$ \Vert g \Vert $ は、 $ g $ の L2 ノルム。

- NOTE: 参考: [勾配消失問題と勾配爆発問題～原因と解決策～ | マサムネの部屋](https://masamunetogetoge.com/gradient-vanish)

page. 64 の `rate` は、 $ \frac{M}{\Vert g \Vert} $ に対応する。
( `threshold` が $ M $ )

よって、 L2 ノルムが上限値 $ M $ を超えたら、
`rate` が 1 より小さくなるので、
$ \frac{M}{\Vert g \Vert} g $ を返却する。


### LSTM の全体図

LSTM の要素は、以下の通り。


#### 主となる流れ ( 図の下部分の、水平方向の現在時間実線上の要素 )

過去の学習を記憶する CEC を中心とした流れである。

- 入力層からの、中間層への、時間 t の入力

$$
\textbf{x}(t) \textbf{W}_a
$$

- 中間層 ( 前回 ) からの、中間層への、時間 t -1 の入力

$$
fx(
  \textbf{u}_a  h(t-1)
  )
$$

- CEC の前の、内積演算ユニットへの入力

$$
a(t) = f(
  \textbf{x}(t) \textbf{W}_a
  fx(
    \textbf{u}_a  h(t-1)
    )
)
$$

$$
i(t)
$$

- CEC への入力
  - NOTE: page. 79 の、「演習チャレンジ」の箇所に該当。<br>
    入力ゲートからと、忘却ゲートの 2 方向からの入力がある。<br>
    `input_gate * a + forget_gate + c`<br>
    `forget_gate` が、 $ f(t) $ である。

$$
i(t) \cdot a(t)
$$

$$
1.0 \times f(t) \times c(t- 1)
$$

- **CEC の演算結果**: CEC の後の、活性化関数への出力

$$
c(t) = i(t) \cdot a(t) + f(t) \cdot c(t - 1)
$$

- CEC の後の、内積演算ユニットの出力

$$
o(t) \cdot g(c(t))
$$

- 出力層への出力

$$
h(t) = o(t) \cdot g(c(t))
$$

- CEC: Constant Error Carousel<br>
  中間層の出力値を記憶する。


- NOTE: 図左上の $ fx $ は、この図全体の、中間層全体のことを表す関数と理解した。<br>
  入力 $ \textbf{x}(t) $ から、出力 $ h(t) $ を出力する、以下の関数。

$$ h(t) = fx( \textbf{x}(t) ) $$

- 黒点線の対象が、「9隠れ層がブロック構造になる(LSTMブロック)」の対象。<br>

- NOTE: 黒点線左側に重なる、 $ \textbf{x}(t) $ は、この「ブロック」のことではなく、
  左側の赤実線の内容と理解した。<br>
  page. 80 のコードの以下に該当する部分と理解した。<br>
  `a, i, f, o = np.hsplit(lstm_in, 4)`


#### 中間層の入力から 入力/忘却/出力ゲートへの流れ ( 赤色現在時間実線 )

記憶することだけを機能とした CEC 代わり、学習を担う各ゲートへの流れである。

- 入力ゲートへの入力

$$
\textbf{x}(t) \textbf{W}_i
$$

- 忘却ゲートへの入力

$$
\textbf{x}(t) \textbf{W}_f
$$

- 出力ゲートへの入力

$$
\textbf{x}(t) \textbf{W}_o
$$


#### 中間層の出力結果からの流れ ( 水色過去時間点線 )

- 中間層 ( 前回 ) からの、中間層への、時間 t -1 の入力 ( 再掲 )

$$
fx(
  \textbf{U}_a  h(t-1)
  )
$$

- 入力ゲートへの入力

$$
h(t -1) \textbf{u}_i
$$

- 忘却ゲートへの入力

$$
h(t -1) \textbf{u}_f
$$

- 出力ゲートへの入力

$$
h(t -1) \textbf{u}_o
$$


#### 覗き穴結合の流れ ( 薄緑現在時間実線 )

- CEC の前の、内積演算ユニットへの入力 ( 再掲 )

$$
i(t) = \sigma (
  \textbf{x} \textbf{W}_i
  h(t -1) \textbf{u}_i
)
$$

- x 1.0 ユニット / 内積演算ユニットへの入力

$$
\sigma(
  \textbf{x}(t) \textbf{W}_f
  h(t -1) \textbf{u}_f
  \textbf{V}_{fc}(t - 1)
)
$$

- CEC への入力: 自己ループ ( 再掲 )

$$
1.0 \times f(t) \times c(t- 1) c(t- 1)
$$

- 出力ゲートへの入力

$$
\textbf{V}_{o} c(t) o(t)
$$

- CEC の後の、内積演算ユニットへの入力

$$
o(t) = \sigma (
  \textbf{x} \textbf{W}_o
  h(t -1) \textbf{u}_o
)
$$


#### 覗き穴結合の流れ ( 薄緑過去時間点線 )

- 入力ゲートへの入力

$$
\textbf{V}_i c(t -1)
$$

- x 1.0 ユニット / 内積演算ユニットへの入力: 自己ループ

$$
1.0 \times f(t) \times c(t- 1) c(t- 1)
$$

- 忘却ゲートへの入力

$$
\textbf{V}_{fc} (t -1)
$$


### 2-1 CEC

RNN では中間層全体で、過去の中間層の出力値を記憶していた。<br>
LSTM では、役割を分離して、過去の中間層の出力値を記憶を CEC が担っている。


#### 数式

$$
\delta^{t-z-1}
=
\delta^{t-z} \{
  \textbf{W} f'(
    \textbf{u^{t-z-1}}
  )
\} = 1
$$

勾配を 1 にして勾配消失と勾配爆発を回避する。

$$
\frac{\partial{E}}
{\partial{c^{t-1}}}
=
\frac{\partial{E}}
{\partial{c^{t}}}
\frac{\partial{c^{t}}}
{\partial{c^{t-1}}} \\
=
\frac{\partial{E}}
{\partial{c^{t}}}
\frac{\partial \{
    a^t - c^{t-1}
  \}
}
{\partial{c^{t-1}}} \\
=
\frac{\partial{E}}
{\partial{c^{t}}}
$$

- NOTE: 以下は -1 では?

$$
\frac{\partial \{
    a^t - c^{t-1}
  \}
}
{\partial{c^{t-1}}}
$$


#### 課題

CEC では時間的依存度に関係なく重みが一律であるため、 NN 内の学習特性が得られない。

CEC の代わりに学習機能を持たせて、 CEC に何を記憶させるかと、どう使うかを制御する必要がある。

- TODO: 入力重み衝突?


### 2-2 入力ゲートと出力ゲート

CEC の課題を解決するため、入力ゲートと出力ゲートを追加する。<br>
それぞれのゲートへの入力値への重みを、重み行列 W, U で可変にする。


- 入力ゲート<br>
  今回の入力値 ( page. 60: 赤実線 ) と、前回の出力値 ( page. 60: 水色点線 ) をもとに、<br>
  CEC に何を記憶させるかを学習する。<br>
  - 今回の入力値に対する重み: $ \textbf{W}_i $
  - 前回の出力値に対する重み: $ \textbf{u}_i $

- 出力ゲート<br>
  今回の入力値 ( page. 60: 赤実線 ) と、前回の出力値 ( page. 60: 水色点線 ) をもとに、<br>
  CEC の出力を、 (出力層への出力として) どう使うかを学習する。
  - 今回の入力値に対する重み: $ \textbf{W}_o $
  - 前回の出力値に対する重み: $ \textbf{u}_o $


### 2-3 忘却ゲート

CEC は、過去の情報を全て保管する。<br>
過去の情報を削除できず、保管され続けてしまう課題がある。

過去の情報が不要となったタイミングで、忘却するための機能として、忘却ゲートを追加する。


### 2-4 覗き穴結合

CEC に保存されている過去の情報を、任意のタイミングで、<br>
他のノードに伝播させたり、忘却させるためのもの。<br>
CEC 自身の値に重み行列を介して、伝播させる構造。

- 他のノードに伝播
  - 出力ゲート ( page. 66 図: 現在時間実線)
  - 入力ゲート ( page. 66 図: 過去時間点線)
- 忘却させる
  - 忘却ゲート ( page. 66 図: 過去時間点線)


実際にはあまり効果が得られなかったとのこと。


## 2. 確認テスト

以降の "page. " は、講義資料のページの番号です。


### page. 62

シグモイド関数の微分は、

$$
f(x) = \frac{1}{1 + e^{x}}
$$

とすると、

$$
f'(x) = f(x) (1 - f(x))
$$

$$
f(0) = 0.5
$$

から、

$$
0.5 \times (1 - 0.5) = 0.25
$$


### page. 78

影響を及ぼさない単語を「忘却」するための、忘却ゲートが作用する。
